In [12]:
import requests
import json

HOST = "http://10.38.2.35:8000/"
API_CANCLED = "canclled"
API_PNR_RANKING = "pnr_ranking?flight_id="
API_ALT_FLIGHT="alt_flight?flight_id="

def get_cancled():
    response = requests.request("GET", HOST + API_CANCLED, headers={}, data={})
    return json.loads(response.text)

def get_pnrs(flight_id):
    response = requests.request("GET", HOST + API_PNR_RANKING + str(flight_id), headers={}, data={})
    return json.loads(response.text)

def get_alt_flights(flight_id):
    response = requests.request("GET", HOST + API_ALT_FLIGHT + str(flight_id), headers={}, data={})
    return json.loads(response.text)

In [13]:
cancel=get_cancled()

In [14]:
cancel['data'][0]
map_cabin={ 'F':'F','P':'F','C':'B','J':'B','Z':'B','Q':'P','R':'P','S':'P','T':'P','H':'P','M':'P',
          'Y':'E', 'A':'E', 'B':'E', 'D':'E', 'E':'E', 'G':'E', 'I':'E', 'K':'E', 'L':'E', 'N':'E', 'O':'E',
           'U':'E', 'V':'E','W':'E','X':'E'}

In [15]:
# get_alt_flights('c53865e9')

In [35]:
pnr={}
alt_flight={}
for f in cancel['data']:
    pnr[f['flight_id']]=get_pnrs(f['flight_id'])['data']
    alt=get_alt_flights(f['flight_id'])['data']
    f2=[]
    for flight in alt:
        f3={}
        f3['flight_id']=flight['flight_id']
        f3['F']=0
        f3['B']=0
        f3['P']=0
        f3['E']=0
#         print()
        for key in map_cabin:
            if key in flight['avilable_seats']:
                f3[map_cabin[key]]+=flight['avilable_seats'][key]
        f2.append(f3)
    alt_flight[f['flight_id']]=f2

In [36]:
for i in alt_flight:
#     print(alt_flight[i])
    break
    

In [37]:
used_seat={}

for i in alt_flight:
    for j in alt_flight[i]:
        used_seat[j['flight_id']]={'F':0,'B':0,'P':0,'E':0}
#             print(j)
        
# used_seat

In [38]:
# pnr

In [39]:
needed_seat=[]
for i in pnr:
    alloc={}
    for j in pnr[i]:
        alloc=j;
        alloc['flight_id']=i
        needed_seat.append(alloc)

In [40]:
sorted_pnr = sorted(needed_seat, key=lambda x: x['score'],reverse=True)

In [41]:
allocated={}

for i in sorted_pnr:
    allocated[i['pnr']]='NULL'
    

In [42]:
index_pnr=0
flight_index={i : {'F':0,'B':0,'P':0,'E':0} for i in pnr}
# flight_index
# used_seat

In [43]:
# allocated

In [44]:
# ik=0
flights=4
cabins=['F','B','P','E']
cabin_ind={'F':0 , 'B':1,'P':2,'E':3}
for i in sorted_pnr:
#     ik+=1
    
#     if(ik>5):
#         break
#     print(i)
    f=0;
    list=[]
    ok=1;
    cabin_no=cabin_ind[map_cabin[i['class']]]
    while(ok==1 and cabin_no<8):
        cabin=cabins[cabin_no%4]
        ind = 0
#     print(used_seat)
        while(ind<len(alt_flight[i['flight_id']])):
        
            max_seat=alt_flight[i['flight_id']][ind][cabin]
            alt = alt_flight[i['flight_id']][ind]['flight_id']
#             print(max_seat)
#             print(ind)
#             print(alt)
#         print(used_seat)
        
            if(used_seat[alt][cabin]+i['pax']<=max_seat):
#                 used_seat[alt][cabin]+=i['pax'];
#                 allocated[i['pnr']]=[alt,cabins[cabin_no%4],1]
#                 print("allovc")
#                 ok=0
#                 break
                
                if(f<flights):
                    if f==0:
                        used_seat[alt][cabin]+=i['pax']
                    list.append([alt,cabins[cabin_no%4]])
                    f+=1;
                else:
                    ok=0
                    break;
        
            ind+=1
            
        cabin_no+=1
    allocated[i['pnr']]=list

In [45]:
# used_seat

In [46]:
# allocated

In [47]:
sum=0
for i in allocated: 
    if allocated[i]=='NULL' or allocated[i]==[]:
        sum+=1

In [48]:
sum


591

In [49]:
len(allocated)

4014

In [31]:
# sorted

<function sorted(iterable, /, *, key=None, reverse=False)>